<a href="https://colab.research.google.com/github/boboguan/QM2Gr13/blob/main/Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Impact of Oil Dependency on the Socio-Economic Development of Major Oil Exporters**



In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
import seaborn as sns
import numpy as np
import plotly
import plotly.express as px
import warnings
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sns.set(font_scale=1.5)
sns.set_style("white")
plt.rcParams['figure.figsize'] = (12, 8)

1. Download Necessary Libraries and Upload Database Into a Data file
2. Use appropriate Data Imputation Techniques to Fill in Missing Data


In [2]:
!mkdir data
!mkdir data/grproject

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Oil Exporting Country Data - COUNTRY DATA.csv to Oil Exporting Country Data - COUNTRY DATA.csv


In [4]:
df = pd.read_csv('Oil Exporting Country Data - COUNTRY DATA.csv', skiprows = 2)

df.columns = ['Country', 'Year', 'GDP per Capita', 'Oil Rent',
              'Debt to GDP Ratio', 'HDI', 'Democracy Index',
              'Civil Rights Freedom Indexes', 'Gini Coefficient']

In [5]:
# Convert 'Year' to integer and other numerical columns to float
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # Convert to numeric, making non-numeric values NaN
df.dropna(subset=['Year'], inplace=True)  # Drop rows where 'Year' is NaN
df['Year'] = df['Year'].astype(int)
df['GDP per Capita'] = pd.to_numeric(df['GDP per Capita'], errors='coerce')
df['Oil Rent'] = pd.to_numeric(df['Oil Rent'], errors='coerce')
df['Debt to GDP Ratio'] = pd.to_numeric(df['Debt to GDP Ratio'], errors='coerce')
df['HDI'] = pd.to_numeric(df['HDI'], errors='coerce')
df['Gini Coefficient'] = pd.to_numeric(df['Gini Coefficient'], errors='coerce')

# Drop completely empty rows if any
df.dropna(how='all', inplace=True)

In [6]:
#Data Imputation
df['Gini Coefficient'] = df['Gini Coefficient'].interpolate(method='linear')
# Replace missing values with the mean or median of the column
df['GDP per Capita'].fillna(df['GDP per Capita'].mean(), inplace=True)
df['Oil Rent'].fillna(df['Oil Rent'].mean(), inplace=True)
df['Debt to GDP Ratio'].fillna(df['Debt to GDP Ratio'].median(), inplace=True)

In [7]:
# Replace missing values with the mode (most frequent value)
# Assuming these are categorical or ordinal and have a common frequent value
df['Democracy Index'].fillna(df['Democracy Index'].mode()[0], inplace=True)
df['Civil Rights Freedom Indexes'].fillna(df['Civil Rights Freedom Indexes'].mode()[0], inplace=True)


In [8]:
# Verify the changes
print(df.head())
print(df.isnull().sum())

        Country  Year  GDP per Capita   Oil Rent  Debt to GDP Ratio    HDI  \
0  Saudi Arabia  2000      8795.26060  41.669171               87.2  0.737   
1  Saudi Arabia  2001      8337.32272  33.352663               93.7  0.744   
2  Saudi Arabia  2002      8380.95930  31.342448               96.9  0.750   
3  Saudi Arabia  2003      9321.80387  37.034522               82.0  0.760   
4  Saudi Arabia  2004     10935.01660  42.640930               65.0  0.770   

  Democracy Index Civil Rights Freedom Indexes  Gini Coefficient  
0           09.08                         8.85               NaN  
1           09.08                         8.85               NaN  
2           09.08                         8.85               NaN  
3           09.08                         8.85               NaN  
4           09.08                         8.85               NaN  
Country                          0
Year                             0
GDP per Capita                   0
Oil Rent                

In [9]:
print(df)

          Country  Year  GDP per Capita   Oil Rent  Debt to GDP Ratio    HDI  \
0    Saudi Arabia  2000      8795.26060  41.669171               87.2  0.737   
1    Saudi Arabia  2001      8337.32272  33.352663               93.7  0.744   
2    Saudi Arabia  2002      8380.95930  31.342448               96.9  0.750   
3    Saudi Arabia  2003      9321.80387  37.034522               82.0  0.760   
4    Saudi Arabia  2004     10935.01660  42.640930               65.0  0.770   
..            ...   ...             ...        ...                ...    ...   
126           UAE  2016     41054.53960  11.080756               19.4  0.870   
127           UAE  2017     43063.96750  13.498573               21.6  0.897   
128           UAE  2018     46722.26870  17.098994               20.9  0.909   
129           UAE  2019     45376.17080  15.681591               27.3  0.920   
130           UAE  2020     37629.17420  10.494098               36.9  0.912   

    Democracy Index Civil Rights Freedo

As seen on the graph, the Gini Coefficienct for Saudi Arabia can not be interpolated as there is only one data point of reference, 2019. Thus this is a weakness.

In [15]:
#Data Cleaning
df = df.replace(r'^\s*$', np.nan, regex=True) #replace the empty string or strings composed with whitespace charater np.nan
df = df.apply(pd.to_numeric, errors='coerce') # change to numeric
df = df.replace(' ', 'NaN', regex=True)

In [17]:
print(df)

     Country  Year  GDP per Capita   Oil Rent  Debt to GDP Ratio    HDI  \
0        NaN  2000      8795.26060  41.669171               87.2  0.737   
1        NaN  2001      8337.32272  33.352663               93.7  0.744   
2        NaN  2002      8380.95930  31.342448               96.9  0.750   
3        NaN  2003      9321.80387  37.034522               82.0  0.760   
4        NaN  2004     10935.01660  42.640930               65.0  0.770   
..       ...   ...             ...        ...                ...    ...   
126      NaN  2016     41054.53960  11.080756               19.4  0.870   
127      NaN  2017     43063.96750  13.498573               21.6  0.897   
128      NaN  2018     46722.26870  17.098994               20.9  0.909   
129      NaN  2019     45376.17080  15.681591               27.3  0.920   
130      NaN  2020     37629.17420  10.494098               36.9  0.912   

     Democracy Index  Civil Rights Freedom Indexes  Gini Coefficient  
0               9.08        